In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("restaurant-review-average") \
    .master("local[*]") \
    .getOrCreate()

In [3]:
sc = spark.sparkContext

In [4]:
sc.defaultParallelism

2

In [5]:
sc.setLogLevel('INFO')

In [6]:
sc.getConf().getAll()

[('spark.driver.host', '67eb04197dfe'),
 ('spark.app.name', 'restaurant-review-average'),
 ('spark.executor.id', 'driver'),
 ('spark.app.id', 'local-1754273750168'),
 ('spark.app.startTime', '1754273749624'),
 ('spark.driver.extraJavaOptions',
  '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=AL

In [5]:
data = [
    (0, "짜장면", "중식", 125),
    (1, "짬뽕", "중식", 235),
    (2, "김밥", "분식", 32),
    (3, "떡볶이", "분식", 534),
    (4, "라멘", "일식", 223),
    (5, "돈가스", "일식", 52),
    (6, "우동", "일식", 12),
    (7, "쌀국수", "아시안", 312),
    (8, "햄버거", "패스트푸드", 12),
    (9, "치킨", "패스트푸드", 23),
]

rdd1 = sc.parallelize(data)
rdd1.take(3)

[(0, '짜장면', '중식', 125), (1, '짬뽕', '중식', 235), (2, '김밥', '분식', 32)]

In [6]:
lines = sc.textFile("file:///home/jovyan/work/learning_spark_data/restaurant_reviews.csv")

lines.take(5)

['id,item,cateogry,reviews,',
 '0,짜장면,중식,125,',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32,',
 '3,떡볶이,분식,534,']

In [7]:
# 총 건수 확인 
print(lines.count())
# 첫 줄, 데이터 분할
header = lines.first()
print(header)
# map으로 split 
# header는 제거
datas = lines.filter(lambda row : row != header)
            #.map(lambda row : row.split(','))
print(datas.take(3))

11
id,item,cateogry,reviews,
['0,짜장면,중식,125,', '1,짬뽕,중식,235,', '2,김밥,분식,32,']


In [9]:
# row? 함수?????
def parse(row):
    fields = row.split(',')

    category = fields[2] # category

    # reviews는 정수로 parse 
    reviews = fields[3]
    reviews = int(reviews)

    return category, reviews

In [10]:
parse('0,짜장면,중식,125,')

('중식', 125)

In [11]:
category_reviews = datas.map(parse)


In [12]:
category_reviews = datas.map(parse)
category_reviews.collect()

[('중식', 125),
 ('중식', 235),
 ('분식', 32),
 ('분식', 534),
 ('일식', 223),
 ('일식', 52),
 ('일식', 12),
 ('아시안', 312),
 ('패스트푸드', 12),
 ('패스트푸드', 23)]

In [13]:
category_review_count = category_reviews.mapValues(lambda x : (x, 1)) # x는 review 개수
category_review_count.collect()

[('중식', (125, 1)),
 ('중식', (235, 1)),
 ('분식', (32, 1)),
 ('분식', (534, 1)),
 ('일식', (223, 1)),
 ('일식', (52, 1)),
 ('일식', (12, 1)),
 ('아시안', (312, 1)),
 ('패스트푸드', (12, 1)),
 ('패스트푸드', (23, 1))]

In [62]:
#같은 key(카테고리)끼리 x와 y의 (리뷰 수 합계, 개수 합계)를 누적
#첫번째 중식 x, 두번째 중식 y , [0] 은 리뷰수, [1] 은 개수 -> 같은 키끼리 행을 바꿔가며 계속 누적해 나간다.
reduced = category_review_count.reduceByKey(lambda x, y : (x[0] + y[0], x[1] + y[1]))
reduced.collect()

[('중식', (360, 2)),
 ('분식', (566, 2)),
 ('일식', (287, 3)),
 ('아시안', (312, 1)),
 ('패스트푸드', (35, 2))]

In [64]:
average = reduced.mapValues(lambda x: round(x[0] / x[1], 1))
average.collect()

[('중식', 180.0), ('분식', 283.0), ('일식', 95.7), ('아시안', 312.0), ('패스트푸드', 17.5)]

map(key,vlaue) -> 

# Narrow Transformation 

- 1:1 변환 → 하나의 열을 다룰 때 다른 데이터가 필요 없는 경우
- filter(), map(), flatMap(), sample(), unio

Narrow vs Wide 
- Narrow: 하나의 파티션에서만 연산됨 (suffle없음)
- Wide: 여러 파티션에서 데이터를 재배치(suffle)함 n()

## flatmap() 
- 요소가 하나하나 떠렁져있었는데 하나의 리스트로 바꾸어 줌 
- 텍스트 분석할떄 댓글 , 리스트로 바꾸어야 할 떄 사용 


In [17]:
rdd_flatmap = rdd.flatMap(lambda x: [x, x + 1])  # => [1, 2, 2, 3, 3, 4]
rdd_flatmap.collect()

[1, 2, 2, 3, 3, 4]

In [21]:
rdd = sc.parallelize([1, 2, 3])
rdd_map = rdd.map(lambda x: [x, x + 1])  # => [[1, 2], [2, 3], [3, 4]]
rdd_map.collect()

[[1, 2], [2, 3], [3, 4]]

movie 예제 (narrow transformation?)

In [18]:
movies = [
    "그린 북",
    "매트릭스",
    "토이 스토리",
    "캐스트 어웨이",
    "포드 V 페라리",
    "보헤미안 랩소디",
    "빽 투 더 퓨처",
    "반지의 제왕",
    "죽은 시인의 사회"
]

In [19]:
moviesRDD = sc.parallelize(movies)
moviesRDD

ParallelCollectionRDD[16] at readRDDFromFile at PythonRDD.scala:289

In [20]:
#job 추가 x
flatMovies = moviesRDD.flatMap(lambda x : x.split(" "))
flatMovies.collect()

['그린',
 '북',
 '매트릭스',
 '토이',
 '스토리',
 '캐스트',
 '어웨이',
 '포드',
 'V',
 '페라리',
 '보헤미안',
 '랩소디',
 '빽',
 '투',
 '더',
 '퓨처',
 '반지의',
 '제왕',
 '죽은',
 '시인의',
 '사회']

In [22]:
# 집합 Transformation

In [23]:
num1 = sc.parallelize([1, 2, 3, 4, 5])
num2 = sc.parallelize([4, 5, 6, 7, 8, 9, 10])

In [24]:
num1.intersection(num2).collect()

[4, 5]

In [26]:
# 합집합 구하기 - union

In [27]:
num_union = num1.union(num2)
num_union

UnionRDD[32] at union at NativeMethodAccessorImpl.java:0

In [28]:
num_union.collect()

[1, 2, 3, 4, 5, 4, 5, 6, 7, 8, 9, 10]

In [29]:
#차집합 구하기 


In [30]:
num1.subtract(num2).collect()

[1, 2, 3]

In [32]:
# 데이터 랜덤 추출 - sample(withReplacement, fraction, seed=None)|

In [36]:
# withReplacement : True -> 중복 추출
num_union.sample(True, 0.3).collect()

[1, 4, 9, 10]

In [37]:

# withReplacement : False -> 중복 X
num_union.sample(False, 0.7).collect()

[1, 3, 4, 5, 4, 5, 7, 8, 9, 10]

In [38]:
# 랜덤을 고정해서 항상 같은 결과가 나올 수 있도록
num_union.sample(True, 0.5, seed=42).collect()

[4, 5, 5, 5, 7]

In [39]:
# Wide Transformations

In [42]:
foods = sc.parallelize([
    "짜장면", "마라탕", "짬뽕", "떡볶이", "쌀국수", "짬뽕", "짜장면", "짜장면", "짜장면", "라면", "우동", "라면"
])
foods

ParallelCollectionRDD[45] at readRDDFromFile at PythonRDD.scala:289

In [43]:
# 그룹핑의 기준을 문자열의 첫 번째 글자로 설정
foodsGroup = foods.groupBy(lambda x : x[0])
foodsGroup

PythonRDD[50] at RDD at PythonRDD.scala:53

In [44]:
res = foodsGroup.collect()

In [45]:
for (k, v) in res:
    print(k, list(v))

짜 ['짜장면', '짜장면', '짜장면', '짜장면']
짬 ['짬뽕', '짬뽕']
쌀 ['쌀국수']
라 ['라면', '라면']
우 ['우동']
마 ['마라탕']
떡 ['떡볶이']


In [46]:
# 내로우 narrow  트랜스포메이션

In [49]:
### 2. Narrow 트랜스포메이션 예제
numbers = sc.parallelize([1, 2, 3, 4, 5])

# 1:1 변환 (Narrow Transformation)
sample_rdd = numbers.sample(False, 0.5)
sample_rdd.take(3)

[1, 5]

In [50]:
union_rdd = numbers.union(sc.parallelize([6,7,8]))
union_rdd.collect()

[1, 2, 3, 4, 5, 6, 7, 8]

In [51]:
kv_rdd = sc.parallelize([('apple',1), ('banana', 2), ('cherry', 3)])

In [53]:
mapp_values = kv_rdd.mapValues(lambda x: x*10)
mapp_values.collect()

[('apple', 10), ('banana', 20), ('cherry', 30)]

In [54]:
# wide 연산 
grouped_rdd = kv_rdd.groupByKey()
grouped_rdd.collect()

[('apple', <pyspark.resultiterable.ResultIterable at 0x7fa5f8a32a50>),
 ('banana', <pyspark.resultiterable.ResultIterable at 0x7fa5f8a31550>),
 ('cherry', <pyspark.resultiterable.ResultIterable at 0x7fa5f8a31250>)]

In [56]:
reduced_rdd = kv_rdd.reduceByKey(lambda x,y: x+y)
reduced_rdd.collect()

[('apple', 1), ('banana', 2), ('cherry', 3)]

In [57]:
### 1. RDD 생성과 기본 트랜스포메이션

# SparkContext 생성 (이미 sc가 존재한다고 가정)
# RDD 생성
numbers = sc.parallelize([1, 2, 3, 4, 5])

# 기본 트랜스포메이션 예제
mapped_rdd = numbers.map(lambda x: x * 2)
filtered_rdd = numbers.filter(lambda x: x % 2 == 0)
flattened_rdd = sc.parallelize([[1, 2], [3, 4]]).flatMap(lambda x: x)

# 결과 확인
print("Mapped RDD:", mapped_rdd.collect())  # [2, 4, 6, 8, 10]
print("Filtered RDD:", filtered_rdd.collect())  # [2, 4]
print("Flattened RDD:", flattened_rdd.collect())  # [1, 2, 3, 4]

Mapped RDD: [2, 4, 6, 8, 10]
Filtered RDD: [2, 4]
Flattened RDD: [1, 2, 3, 4]


In [59]:
### 4. 액션 연산 예제
# 액션 연산
count = numbers.count()
first_element = numbers.first()
collected = numbers.collect()
taken = numbers.take(3)

In [60]:
### 2. Narrow 트랜스포메이션 예제

# 1:1 변환 (Narrow Transformation)
sample_rdd = numbers.sample(False, 0.5)
union_rdd = numbers.union(sc.parallelize([6, 7, 8]))

# Key-Value RDD에서 mapValues 사용
kv_rdd = sc.parallelize([("apple", 1), ("banana", 2), ("cherry", 3)])
mapped_values = kv_rdd.mapValues(lambda x: x * 10)

print("Sample RDD:", sample_rdd.collect())  # 무작위 샘플링된 결과
print("Union RDD:", union_rdd.collect())  # [1, 2, 3, 4, 5, 6, 7, 8]
print("MapValues RDD:", mapped_values.collect())  # [("apple", 10), ("banana", 20), ("cherry", 30)]


Sample RDD: [4]
Union RDD: [1, 2, 3, 4, 5, 6, 7, 8]
MapValues RDD: [('apple', 10), ('banana', 20), ('cherry', 30)]


In [61]:
### 3. Wide 트랜스포메이션 예제

# 셔플링이 필요한 변환 (Wide Transformation)
distinct_rdd = sc.parallelize([1, 1, 2, 2, 3, 3]).distinct()
grouped_rdd = kv_rdd.groupByKey()
reduced_rdd = kv_rdd.reduceByKey(lambda x, y: x + y)

# join ()

In [65]:
# 두 RDD 조인하기
rdd1 = sc.parallelize([("apple", 2), ("banana", 1)])
rdd2 = sc.parallelize([("apple", "fruit"), ("banana", "fruit"), ("carrot", "vegetable")])

# 두 RDD 조인
joined_rdd = rdd1.join(rdd2)
print(joined_rdd.collect())
# 출력 결과: [('apple', (2, 'fruit')), ('banana', (1, 'fruit'))]

print("Distinct RDD:", distinct_rdd.collect())  # [1, 2, 3]
print("Joined RDD:", joined_rdd.collect())  # [("apple", (2, "fruit")), ("banana", (1, "fruit"))]

[('apple', (2, 'fruit')), ('banana', (1, 'fruit'))]
Distinct RDD: [2, 1, 3]
Joined RDD: [('apple', (2, 'fruit')), ('banana', (1, 'fruit'))]


In [66]:
# 왼쪽 외부 조인 (leftOuterJoin)
left_joined = rdd1.leftOuterJoin(rdd2)
print(left_joined.collect())
# 출력 결과: [('apple', (2, 'fruit')), ('banana', (1, 'fruit'))]

# 오른쪽 외부 조인 (rightOuterJoin)
right_joined = rdd1.rightOuterJoin(rdd2)
print(right_joined.collect())
# 출력 결과: [('apple', (2, 'fruit')), ('banana', (1, 'fruit')), ('carrot', (None, 'vegetable'))]

# subtractByKey 연산
subtract_result = rdd1.subtractByKey(sc.parallelize([("apple", "any")]))
print(subtract_result.collect())
# 출력 결과: [('banana', 1)]

[('apple', (2, 'fruit')), ('banana', (1, 'fruit'))]
[('apple', (2, 'fruit')), ('banana', (1, 'fruit')), ('carrot', (None, 'vegetable'))]
[('banana', 1)]


# reduce()

In [67]:
# reduce 연산 (결합성과 교환성을 가진 작업)
sum_result = numbers.reduce(lambda x, y: x + y)
product_result = numbers.reduce(lambda x, y: x * y)

print("Count:", count)  # 5
print("First element:", first_element)  # 1
print("Collected:", collected)  # [1, 2, 3, 4, 5]
print("Taken:", taken)  # [1, 2, 3]
print("Sum:", sum_result)  # 15
print("Product:", product_result)  # 120

Count: 5
First element: 1
Collected: [1, 2, 3, 4, 5]
Taken: [1, 2, 3]
Sum: 15
Product: 120


In [68]:
# 파티션을 3개로 지정ㄴ 
sc.parallelize([1,2,3,4],3).reduce(lambda x,y : (x*2)+y)

18

In [69]:
spark.stop()

In [70]:
sc.stop()